In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Reading CSV files

In [2]:
members = pd.read_csv("members.csv")
songs = pd.read_csv("songs.csv")
songsextrainfo = pd.read_csv("song_extra_info.csv")
datatrain = pd.read_csv("train.csv")